In [31]:
import sys
sys.path.append('../')

In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
%reload_ext autoreload

In [34]:
from scripts.detect_voids import get_neightbours, search_voids_bb_neightbours
from packages.yolo_predict import YOLO_Pred
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import numpy as np
import cv2
import multiprocessing
from functools import partial
import itertools
import os

In [35]:
yolo = YOLO_Pred('/home/cremerf/FinalProject/data/first_training/weights/bestnoft.onnx', '/home/cremerf/FinalProject/data/config_blmodel.yaml')

In [43]:
img_path = '/home/cremerf/FinalProject/data/images/test/test_2097.jpg'

In [44]:
image = cv2.imread(img_path)
h_image, w_image = image.shape[0:2] # limits of the image
df_predictions = yolo.predictions(image=image)

# Get neightbours from 3 ways (right / left / up)
pool = multiprocessing.Pool()
neightbour = 'left'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_left = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

# clean up empty positions
dict_of_neightbours_left = list(filter(None, dict_of_neightbours_left))

pool = multiprocessing.Pool()
neightbour = 'right'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_right = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_right = list(filter(None, dict_of_neightbours_right))

'''
pool = multiprocessing.Pool()
neightbour = 'up'
func = partial(get_neightbours, df_predictions, neightbour)
dict_of_neightbours_up = pool.map(func, list(df_predictions.index))
pool.close()
pool.join()

dict_of_neightbours_up = list(filter(None, dict_of_neightbours_up))
'''

# Merged 3 separated list of dicts(left/right/high) into 1 list
dicts_neightbours = [dict_of_neightbours_left, dict_of_neightbours_right]
list_neightbours = list(itertools.chain.from_iterable(dicts_neightbours))

# Get void neightbours 
list_of_voids = search_voids_bb_neightbours(df_predictions=df_predictions, list_of_dicts=list_neightbours, h_image=h_image, w_image=w_image, img_path= img_path)

# Create dataframe with data(X_center/Y_center/Label) of voids
df_voids = pd.DataFrame(list_of_voids, columns=['Neightbour','Label','x1','y1','x2','y2', 'Width','Height'])

Evaluating 173...
Evaluating 181...
Evaluating 233...
Evaluating 179...
Evaluating 556...
Evaluating 438...
Evaluating 501...
Evaluating 11...
Evaluating 174...
Evaluating 569...
Evaluating 554...
Evaluating 549...
Evaluating 299...
Evaluating 521...
Evaluating 441...
Evaluating 546...
Evaluating 878...
Evaluating 402...
Evaluating 505...
Evaluating 319...
Evaluating 891...
Evaluating 177...
Evaluating 175...
Evaluating 192...
Evaluating 500...
Evaluating 164...
Evaluating 828...
Evaluating 474...
Evaluating 557...
Evaluating 334...
Evaluating 791...
Evaluating 471...
Evaluating 560...
Evaluating 513...
Evaluating 657...
Evaluating 816...
Evaluating 571...
Evaluating 516...
Evaluating 656...
Evaluating 580...
Evaluating 170...
Evaluating 581...
Evaluating 526...
Evaluating 662...
Evaluating 559...
Evaluating 203...
A la izquierda de 203 hay espacio vacio
Evaluating 72...
Evaluating 562...
Evaluating 813...
Evaluating 558...
Evaluating 801...
Evaluating 563...
Evaluating 110...
Evaluati

In [45]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,203,203 void #1,239.126488,1587.569778,346.302681,1724.415817,107.176193,136.846039
1,170,170 void #1,522.468880,1284.883480,606.483604,1410.608707,84.014725,125.725227


In [9]:
df_voids

,Neightbour,Label,x1,y1,x2,y2,Width,Height
0,799,799 void #1,678.953061,1940.094954,812.004424,2251.455278,133.051363,311.360325
1,54,54 void #1,830.681572,1622.041496,971.960762,1835.113075,141.279190,213.071579
2,367,367 void #1,352.162576,909.913575,452.828296,1148.631743,100.665720,238.718168
3,530,530 void #1,136.416377,2842.296151,222.232350,3125.644095,85.815973,283.347944
4,405,405 void #1,1767.942859,1305.920386,1981.513993,1367.453958,213.571135,61.533572
5,413,413 void #1,88.934991,1363.597500,354.394992,1452.077412,265.460001,88.479912
6,661,661 void #1,1658.429095,1996.015251,1831.768672,2245.846107,173.339578,249.830856
7,472,472 void #1,1681.495659,1947.390890,1824.133707,2277.309977,142.638047,329.919086
8,275,275 void #1,2156.311114,2358.441194,2301.435174,2679.007628,145.124060,320.566435
9,424,424 void #1,785.095965,1608.560190,933.692986,1802.401693,148.597021,193.841503


In [46]:
#image_path_bb = os.path.join(folder_path_final, filename)

img_path = '/home/cremerf/FinalProject/scripts/test_neightbours2.jpg'
# load the image
image = cv2.imread(img_path)

# loop over indexes and for each index plot the rectangles
#for idx in df_voids.index:

# get the coordinates for each index/rectangle
for i in df_voids.index:
    x1 = int(df_voids.loc[i][2]) 
    y1 = int(df_voids.loc[i][3]) 
    x2 = int(df_voids.loc[i][4])  
    y2 = int(df_voids.loc[i][5])


    # # Window name in which image is displayed
    window_name = 'Object'
    # represents the top left corner of rectangle
    start_point=(x1, y1)

    # represents the top right corner of rectangle
    end_point=(x2,y2)

    # # Blue color in BGR
    color = (0, 0, 255)

    # # Line thickness of 2 px
    thickness = 5

    # plot the rectangle over the image
    image = cv2.rectangle(image, start_point, end_point, color, thickness)

# save the img
cv2.imwrite(filename='test_voids5.jpg', img=image)

True

In [ ]:
9	424	424 void #1	785.095965	1608.560190

In [17]:
def image_mean(x, y, w, h, img_path):
    
    image = cv2.imread(img_path)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    roi = np.mean(gray[y:y + h, x:x + w])

    return roi

In [42]:
df_voids.loc[6]

Neightbour            661
Label         661 void #1
x1                1658.43
y1                1996.02
x2                1831.77
y2                2245.85
Width              173.34
Height            249.831
Name: 6, dtype: object

In [43]:
roi = image_mean(x=1658,y=1996,w=173,h=249,img_path=img_path)

In [44]:
roi

38.753441511711586

In [45]:
df_predictions.loc[661]

X_center    1918.438461
Y_center    2120.930679
Width        173.339578
Height       249.830856
Name: 661, dtype: float64

In [46]:
roi = image_mean(x=int((1918-173/2)),y=int((2120-249/2)),w=173,h=249,img_path=img_path)

In [47]:
roi

91.66453095619472

In [ ]:
roi = image_mean(x=int((2083-145/2)),y=int((2518-320/2)),w=145,h=320,img_path=img_path)